In [7]:
import numpy as np
import chainer
import chainer.functions as F
import chainer.links as L
import chainer.initializers as I
from chainer import training
from chainer.training import extensions
import cv2

In [12]:
import sys
import time

In [8]:
class MyChain(chainer.Chain):
    def __init__(self):
        super(MyChain, self).__init__()
        with self.init_scope():
            self.conv1 = L.Convolution2D(1, 4, 3, 1, 1)
            self.conv2 = L.Convolution2D(4, 8, 3, 1, 1)
            self.l3 = L.Linear(None, 10)
    def __call__(self, x):
        h1 = F.max_pooling_2d(F.relu(self.conv1(x)), ksize=2, stride=2)
        h2 = F.max_pooling_2d(F.relu(self.conv2(h1)), ksize=2, stride=2)
        return self.l3(h2)

In [9]:
model = L.Classifier(MyChain(), lossfun=F.softmax_cross_entropy)
chainer.serializers.load_npz('result/mnist.model', model)

In [13]:
cap = cv2.VideoCapture(1)
while True:
    ret, frame = cap.read()            
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    xp = int(frame.shape[1]/2)
    yp = int(frame.shape[0]/2)
    d = 28*2
    cv2.rectangle(gray, (xp-d, yp-d), (xp+d, yp+d), color=0, thickness=2)
    cv2.startWindowThread()
    cv2.imshow('gray', gray)
    if cv2.waitKey(10) == 27:
        cv2.waitKey(1)
        cv2.destroyAllWindows()
        cv2.waitKey(1)
        break
    gray = cv2.resize(gray[yp-d:yp + d, xp-d:xp + d],(28, 28))
    img = np.zeros((28,28), dtype=np.float32)
    img[np.where(gray>64)]=1
    img = 1-np.asarray(img,dtype=np.float32)  # 0〜1に正規化してみる
    img = img[np.newaxis, np.newaxis, :, :] # 4次元行列に変換（1x1x8x8，バッチ数xチャンネル数x縦x横）
    x = chainer.Variable(img)
    y = model.predictor(x)
    c = F.softmax(y).data.argmax()
#     print(c)
    sys.stdout.write("\r%d" % c)
    sys.stdout.flush()
    time.sleep(0.01)
                    
cap.release()


1